In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup as Soup
import pyLDAvis.sklearn
# import pandas as pd
import os

ABSTRACT_PATH = './all_abstracts'
NUM_TOPICS = 15


In [3]:
def getCorpus():
    abstractList = []
    idList = []
    for subdir, dirs, files in os.walk(ABSTRACT_PATH):
        for file in files:
            if file.endswith('xml'):
                file = open(subdir + "/" + file, encoding="utf8").read()
                soup = Soup(file)
                abstracts = soup.findAll('ce:para')
                ids = soup.findAll('itemid', {'idtype' : 'PUI'})
                if len(abstracts) > 0:
                    abstractList.append(abstracts[0].get_text())
                    idList.append(ids[0].get_text())
    return (abstractList, idList)

# def getCorpus():
#     corpus = []
#     for fileName in os.listdir(ABSTRACT_PATH):
#         f = open(ABSTRACT_PATH+'/'+fileName, "r",encoding="utf8")
#         corpus.append(f.read())
#     return corpus

In [5]:
import numpy as np
def displayLDAModel(abstractList):
    data = abstractList
    # NUM_TOPICS = 5
    stemmer = SnowballStemmer("english")
    analyzer = CountVectorizer(stop_words='english', lowercase=True).build_analyzer()

    def stemmed_words(doc):
        return (stemmer.stem(w) for w in analyzer(doc))

    vectorizer = CountVectorizer(analyzer=stemmed_words,
                                 stop_words='english', lowercase=True, 
                                 token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
    data_vectorized = vectorizer.fit_transform(data)
     
    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
    lda_Z = lda_model.fit_transform(data_vectorized)
     
    # text = "The economy is working better than ever"
    # x = lda_model.transform(vectorizer.transform([text]))[0]
    # print(x, x.sum())
    # pyLDAvis.enable_notebook()
    def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
        for topic_idx, topic in enumerate(H):
            print ("Topic %d:" % (topic_idx))
            print (" ".join([feature_names[i]
                            for i in topic.argsort()[:-no_top_words - 1:-1]]))
            top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
            for doc_index in top_doc_indices:
                print (documents[doc_index])
    
    
    lda_W = lda_model.transform(data_vectorized)
    lda_H = lda_model.components_
    
    no_top_words = 4
    no_top_documents = 4
    display_topics(lda_H, lda_W, vectorizer.get_feature_names(), abstractList, no_top_words, no_top_documents)
    panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
    pyLDAvis.show(panel)
    print(panel)    
displayLDAModel(getCorpus()[0])

<input>:13: DeprecationWarning: invalid escape sequence \-
<input>:13: DeprecationWarning: invalid escape sequence \-
<input>:13: DeprecationWarning: invalid escape sequence \-
<ipython-input-5-623ac11a919f>:13: DeprecationWarning: invalid escape sequence \-
  token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')


C:\Users\wongr\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


Topic 0:
express cell gene cancer
Connexin 31 (Cx31) is considered a suppressor for many tumors. Ginsenoside (Rg1) is a traditional Chinese herb that is widely acknowledged due to its anti-tumor characteristics. However, limited studies have focused on the role of Rg1 in papillary thyroid cancer (PTC) cells. In the current study, we found that the expression of Cx31 in thyroid cancer tissues and thyroid cancer cell lines was significantly lower than that in normal thyroid epithelial tissues and cell lines. Overexpression of Cx31 reduced thyroid cancer cell proliferation, migration and invasion. Furthermore, we found that Rg1 significantly enhanced the expression of Cx31. Moreover, the proliferation and migration of IHH-4 and BCPAP cells were significantly reduced by Rg1 treatment. In contrast, the silencing of Cx31 enhanced the expression of Ki67 and proliferating cell nuclear antigen (PCNA). Meanwhile, treatment with Rg1 significantly decreased the protein levels of Ki67 and PCNA, but

C:\Users\wongr\AppData\Local\Programs\Python\Python37-32\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [18/Jan/2019 10:20:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2019 10:20:10] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2019 10:20:11] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2019 10:20:11] "GET /LDAvis.js HTTP/1.1" 200 -


In [44]:
def displayTopicDocuments(ldaModel, idList):
    documentToTopics = pd.DataFrame(ldaModel)
    documentIds = pd.DataFrame({'document_id':idList})
    documentToTopicsWithId = documentToTopics.join(documentIds)
    print(documentToTopicsWithId)

In [45]:
import numpy as np
corpus = getCorpus()

# abstractList = corpus[0]
abstractList = corpus
# idList = corpus[1]
# print(abstractList)

# lda_model = displayLDAModel(abstractList)
# displayTopicDocuments(lda_model, idList)
stemmer = SnowballStemmer("english")
analyzer = CountVectorizer(stop_words='english', lowercase=True).build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(analyzer=stemmed_words)

data_vectorized = vectorizer.fit_transform(abstractList)

lda_model = LatentDirichletAllocation(n_topics=4, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
lda_model = lda_model.transform(vectorizer.transform(abstractList))

for att in dir(lda_model):
    print (att, getattr(lda_model,att))
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
# print(panel)
print(panel)
pyLDAvis.show(panel)




C:\Users\wongr\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


T [[1.65001006e-04 2.27168186e-04 1.74639861e-04 2.06753606e-04
  2.10498757e-04 2.25151132e-04 2.24036527e-04 3.23828882e-04
  3.23828882e-04 3.20500646e-04 3.20500646e-04]
 [1.63993846e-04 2.26118535e-04 1.73362792e-04 2.05627630e-04
  2.09476827e-04 2.23992511e-04 2.22822384e-04 3.20606708e-04
  3.20606708e-04 3.18832581e-04 3.18832581e-04]
 [1.64516386e-04 2.26866910e-04 1.73836374e-04 2.06413458e-04
  2.09980975e-04 2.24795339e-04 2.23551601e-04 3.21619785e-04
  3.21619785e-04 3.19201203e-04 3.19201203e-04]
 [9.99506489e-01 9.99319846e-01 9.99478161e-01 9.99381205e-01
  9.99370043e-01 9.99326061e-01 9.99329589e-01 9.99033945e-01
  9.99033945e-01 9.99041466e-01 9.99041466e-01]]
__abs__ <method-wrapper '__abs__' of numpy.ndarray object at 0x11AB2CF0>
__add__ <method-wrapper '__add__' of numpy.ndarray object at 0x11AB2CF0>
__and__ <method-wrapper '__and__' of numpy.ndarray object at 0x11AB2CF0>
__array__ <built-in method __array__ of numpy.ndarray object at 0x11AB2CF0>
__array_finali

AttributeError: 'numpy.ndarray' object has no attribute 'components_'